# 实验六  线性代数方程组的直接解法

顺序高斯消去法：

In [1]:
import numpy as np

def gaussian_elimination_sequence(a, b):
    """
    用「顺序高斯消去法」解线性方程 `ax = b`。
    
    Args:
        a : np_array_like 系数矩阵（nxn）
        b : np_array_like 右端常数（n）
    
    Returns:
        x : np.array `ax=b` 的解（n）
        
    Raises:
        Exception("求解失败") if a[k][k] == 0
    """
    A = np.c_[a, b]  # 增广矩阵
    
    n, c = A.shape
    assert c == n + 1, f'bad shape: {A.shape}'
    
    x = np.zeros(n)

    # 消元
    for k in range(n-1):
        if A[k][k] == 0:
            raise Exception("求解失败")

        for i in range(k+1, n):
            m = A[i][k] / A[k][k];
            A[i][k] = 0;
            for j in range(k+1, n+1):
                A[i][j] -= A[k][j] * m

    # 回代
    x[n-1] = A[n-1][n] / A[n-1][n-1]
    for k in range(n-2, -1, -1): # from n-2 (included) to 0 (included)
        for j in range(k+1, n):
            A[k][n] -= A[k][j] * x[j]
        x[k] = A[k][n] / A[k][k]
        
    return x

In [2]:
q = np.array([[0.101, 2.304, 3.555, 1.183],
              [-1.347, 3.712, 4.623, 2.137], 
              [-2.835, 1.072, 5.643, 3.035]])
a, b = q[:, :-1],  q[:, -1]
gaussian_elimination_sequence(a, b)

array([-0.39823377,  0.01379507,  0.33514424])

In [3]:
# 用 numpy 计算检查
np.linalg.solve(a, b)

array([-0.39823377,  0.01379507,  0.33514424])

列主元高斯消去算法

In [4]:
def gaussian_elimination(a, b, sequence=False):
    """
    用「列主元高斯消去法」解线性方程 `ax = b`。
    
    本函数可以通过「列主元高斯消元法」或「顺序高斯消元法」计算，
    通过参数 sequence 控制，默认 sequence=False 使用「列主元高斯消元法」。
    
    Args:
        a : np_array_like 系数矩阵（nxn）
        b : np_array_like 右端常数（n）
    
    Returns:
        x : np.array `ax=b` 的解（n）
        
    Raises:
        Exception("求解失败") if a[k][k] == 0
    """
    A = np.c_[a, b]  # 增广矩阵
    
    n, c = A.shape
    assert c == n + 1, f'bad shape: {A.shape}'
    
    x = np.zeros(n)
    
    # 消元
    for k in range(0, n-1):
        if not sequence:  # 列主元
            i_max = k + np.argmax(np.abs(A[k:n, k]))

            if A[i_max][k] == 0:
                raise Exception("A 奇异，求解失败")

            A[[i_max, k]] = A[[k, i_max]]  # swap rows
        
        for i in range(k+1, n):
            m = A[i][k] / A[k][k];
            for j in range(k+1, n+1):
                A[i][j] -= A[k][j] * m
    
    # 回代
    if a[n-1][n-1] == 0:
        raise Exception("矩阵奇异，求解失败")
    
    x[n-1] = A[n-1][n] / A[n-1][n-1]
    for i in range(n-2, -1, -1): # from n-2 (included) to 0 (included)
        for j in range(i+1, n):
            A[i][n] -= A[i][j] * x[j]
        x[i] = A[i][n] / A[i][i]
        
    return x

In [5]:
q = np.array([[0.101, 2.304, 3.555, 1.183],
              [-1.347, 3.712, 4.623, 2.137], 
              [-2.835, 1.072, 5.643, 3.035]])
a, b = q[:, :-1],  q[:, -1]
print('列主元高斯消元:\t', gaussian_elimination(a, b))
print('顺序高斯消元:\t', gaussian_elimination(a, b, sequence=True))

列主元高斯消元:	 [-0.39823377  0.01379507  0.33514424]
顺序高斯消元:	 [-0.39823377  0.01379507  0.33514424]


LU 分解

In [6]:
def lu(a, sequence=False, swap_times: list = {}):
    """
    「高斯消去法」的 LU 分解.
    
    本函数可以计算「列主元高斯消元法」、「顺序高斯消元法」的 LU 分解，
    通过参数 sequence 控制，默认 sequence=False 使用「列主元高斯消元法」。
    
    Args:
        a: np_array_like 方阵 (nxn)
        sequence: bool, True 则使用顺序高斯消去法，False 为列主元的高斯消去法
            default: sequence=False
        swap_times: 这是一个**输出**用的变量，只有传入 dict 变量时才有效。
            若使用「列主元高斯消元法」（sequence=False）
            则，置 swap_times['swap_times'] = 行交换次数。
            这个值正常的输出中不需要，但在一些问题，比如，
            利用 LU 分解求行列式时，得到 swap_times 会很有帮助。
    
    Returns:
        (l, u, p): result
        
        l: np.array, Lower triangle result (nxn)
        u: np.array, Upper triangle result (nxn)
        p: np.array, Permutation: 交换后的行顺序 (n)
            p = None if sequence=True
        
    Raises:
        Exception: 存在为零的主元素
    """
    a = np.array(a, dtype=np.float)  # copy
    
    assert a.shape[0] == a.shape[1]
    n = a.shape[0]
    
    
    if not sequence:
        # p 记录行交换的过程，使用「列主元高斯消元法」才使用，否则为 None
        p = np.array([k for k in range(n)])
        # swap_times:  行交换次数
        if isinstance(swap_times, dict):
            swap_times['swap_times'] = 0
    else:
        p = None
        
    
    for k in range(n-1):
        if not sequence:
            i_max = k + np.argmax(np.abs(a[k:n, k]))

            if i_max != k:
                a[[i_max, k]] = a[[k, i_max]]  # swap rows
                p[[i_max, k]] = p[[k, i_max]]  # record
                swap_times['swap_times'] += 1
        
        if a[k][k] == 0:
            raise Exception("存在为零的主元素")
            
        for i in range(k+1, n):
            a[i][k] /= a[k][k]  # L @ 严格下三角
            for j in range(k+1, n):
                a[i][j] -= a[i][k] * a[k][j]  # U @ 上三角
    
    # print(a, p)
    
    # Uncommit the following lines to get a Permutation Matrix
    # Only for sequence=False
    # pm = np.zeros_like(a)
    # for i, v in enumerate(p):
    #     pm[i, v] = 1
    #     print(pm)
    
    return np.tril(a, k=-1) + np.identity(a.shape[0]), np.triu(a), p

In [7]:
# 列主元高斯消元
lu([[1,2,2], [4,4,2], [4,6,4]])

(array([[1.  , 0.  , 0.  ],
        [1.  , 1.  , 0.  ],
        [0.25, 0.5 , 1.  ]]),
 array([[4. , 4. , 2. ],
        [0. , 2. , 2. ],
        [0. , 0. , 0.5]]),
 array([1, 2, 0]))

In [8]:
# 顺序高斯消元
lu([[1,2,2], [4,4,2], [4,6,4]], sequence=True)

(array([[1. , 0. , 0. ],
        [4. , 1. , 0. ],
        [4. , 0.5, 1. ]]),
 array([[ 1.,  2.,  2.],
        [ 0., -4., -6.],
        [ 0.,  0., -1.]]),
 None)

In [9]:
def solve_lu(b, l, u, p=None):
    """用 lu(a) 得到的 `pa=lu` 分解的结果求解原方程组 `ax=b` 的解 x。
    
    若 p 不为 None 则使用「列主元高斯消元」，p 为 None表示使用「顺序高斯消元」。
        
        # `@` means matrix multiplication, refer: https://docs.python.org/reference/expressions.html#binary-arithmetic-operations
        b = p @ b if p != None
        l @ y = b
        u @ x = y
    
    Args:
        b: np_array_like, 原方程组的右端常数（n）
        l: np_array_like, Lower triangle of lu_seq(a)
        u: np_array_like, Upper triangle of lu_seq(a)
        p: np_array_like, LU分解中交换后的行顺序
            default p=None: 未做行交换，即使用顺序高斯消去法
        
        使用列主元高斯消元法时，l, u, p 使用 lu(a) 得到的结果即可：
            solve_lu(b, *lu(a))
        或者使用顺序高斯消元：
            solve_lu(b, *lu(a, sequence=True))  # p=None
        
    Returns:
        x : np.array `ax=b` 的解（n）
    """
    assert np.shape(l) == np.shape(u)
    assert np.shape(l)[0] == np.shape(b)[0]
    
    n = np.shape(l)[0]
    
    # do swap
    if p is not None:
        b = [b[v] for v in p]

    # L * y = b
    y = np.zeros(n, dtype=np.float)
    y[0] = b[0]
    for i in range(1, n):
        bi = b[i]
        for j in range(0, i):
            bi -= y[j] * l[i][j]
        y[i] = bi / l[i][i]
    # print(y)

    # U * x = y
    x = np.zeros(n, dtype=np.float)
    x[n-1] = y[n-1] / u[n-1][n-1]
    for i in range(n-2, -1, -1): # from n-2 (included) to 0 (included)
        yi = y[i]
        for j in range(i+1, n):
            yi -= x[j] * u[i][j]
        x[i] = yi / u[i][i]
    # print(x)
    
    return x

In [10]:
q = np.array([[0.101, 2.304, 3.555, 1.183],
              [-1.347, 3.712, 4.623, 2.137], 
              [-2.835, 1.072, 5.643, 3.035]])
a, b = q[:, :-1],  q[:, -1]

# 列主元高斯消元
x = solve_lu(b, *lu(a)); print(x)
# 顺序高斯消元
x = solve_lu(b, *lu(a, sequence=True)); print(x)

[-0.39823377  0.01379507  0.33514424]
[-0.39823377  0.01379507  0.33514424]


LU 分解求行列式

In [11]:
def det(a):
    """矩阵行列式
    
    利用 LU 分解（列主元高斯消元）求方阵 a 的行列式: d = det(a)
    
    Args:
        a: np_array_like, 要求行列式的矩阵
        
    Returns:
        d: float, a 的行列式值。
        
    Reference:
        https://blog.csdn.net/nstarLDS/article/details/106074256
    """
    swap_times = {}
    l, u, p = lu(a, sequence=False, swap_times=swap_times)
    sign = -1 if swap_times['swap_times'] % 2 == 1 else 1
    return np.prod(np.diag(u)) * sign

In [12]:
a = [[ 0.101, 2.304, 3.555],
     [-1.347, 3.712, 4.623],
     [-2.835, 1.072, 5.643]]
print(np.linalg.det(a))
print(det(a))

21.20912390399999
21.209123904000002


In [13]:
a = [[1,2], [3,4]]
print(np.linalg.det(a))
print(det(a))

-2.0000000000000004
-2.0


In [14]:
def test_det(dim):
    a = np.random.random((dim, dim))
    print('np.det:', np.linalg.det(a))
    s = {}
    l, u, p = lu(a, sequence=False, swap_times=s)
    print('pd dia:', np.prod(np.diag(u)))
    print('my det:', det(a))
    print(s)
for i in range(5):
    test_det(3)
    print('----')

np.det: -0.01943799105427653
pd dia: -0.01943799105427655
my det: -0.01943799105427655
{'swap_times': 2}
----
np.det: 0.0476193547152672
pd dia: 0.047619354715267216
my det: 0.047619354715267216
{'swap_times': 2}
----
np.det: 0.0017777819144817564
pd dia: -0.001777781914481734
my det: 0.001777781914481734
{'swap_times': 1}
----
np.det: -0.058467358804227446
pd dia: 0.058467358804227446
my det: -0.058467358804227446
{'swap_times': 1}
----
np.det: -0.0026315646257671272
pd dia: -0.002631564625767127
my det: -0.002631564625767127
{'swap_times': 2}
----


LU 分解求逆

In [15]:
def inv(a):
    """矩阵的逆
    
    利用 LU 分解（列主元高斯消元）求方阵 a 的逆矩阵: x = inv(a)
    
    Args:
        a: np_array_like, 待求逆的矩阵
        
    Returns:
        x: float, a 的逆矩阵
        
    Reference:
        https://en.wikipedia.org/wiki/LU_decomposition#Inverting_a_matrix
    """
    l, u, p = lu(a)
    
    X = np.zeros_like(a, dtype=np.float)
    B = np.identity(np.shape(a)[0])
    
    for i, b in enumerate(B.T):  # iter on cols
        x = solve_lu(b, l, u, p)
        X[:, i] = x
        
    return X

In [16]:
inv(np.identity(3))

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [17]:
a = [[1,2], [3,4]]
print(np.linalg.inv(a))
inv(a)

[[-2.   1. ]
 [ 1.5 -0.5]]


array([[-2. ,  1. ],
       [ 1.5, -0.5]])

In [18]:
# 多尝试一些，看看是否正确

err = 0

for i in range(100):
    a = np.random.random((i, i))
    
    inv_np = np.linalg.inv(a)
    inv_my = inv(a)

    if not np.all(inv_my - inv_np < 1e-8):
        err += 1
    
print(err)

0


In [19]:
# 测试运行速度

import time

def time_cost(func):
    """print time cost of a function func"""
    def wrapper(*args, **kwargs):
        s = time.time()
        ret = func(*args, **kwargs)
        e = time.time() - s
        print(f'{func.__name__} executed in {e: .3f} s')
        return ret
    return wrapper

@time_cost
def np_inv(a):
    return np.linalg.inv(a)

@time_cost
def my_inv(a):
    return inv(a)

In [20]:
a = np.random.random((150, 150))

inv_np = np_inv(a)
inv_my = my_inv(a)

print(np.all(inv_my - inv_np < 1e-8))

np_inv executed in  0.005 s
my_inv executed in  3.866 s
True
